In [1]:
import requests
import pandas as pd

# Replace with your actual API key
API_KEY = "391d2c632b433f4ed3a318ad2bdb814e"
SPORT = "basketball_ncaab"  # Sport key for NCAAB
REGION = "us"  # Odds for the US market
#BOOKMAKERS = ["draftkings", "fanduel", "espn"]  # Add/remove sportsbooks

# API URL
URL = f"https://api.the-odds-api.com/v4/sports/{SPORT}/odds/"

# Request Parameters
params = {
    "api_key": API_KEY,
    "regions": REGION,
    "markets": "spreads" #"h2h,spreads,totals"  # Moneyline, spreads, and over/under
    #"bookmakers": ",".join(BOOKMAKERS),
}

# Make API request
response = requests.get(URL, params=params)

if response.status_code == 200:
    odds_data = response.json()

    # Convert to DataFrame
    games = []
    for game in odds_data:
        for bookmaker in game.get("bookmakers", []):
            for market in bookmaker.get("markets", []):
                for outcome in market.get("outcomes", []):
                    games.append({
                        "Game": f"{game['home_team']} vs {game['away_team']}",
                        "Bookmaker": bookmaker["key"],
                        "Market": market["key"],
                        "Team": outcome.get("name"),
                        "Odds": outcome.get("price"),
                        "Point": outcome.get("point")  # For spreads/totals
                    })

    df = pd.DataFrame(games)
    print(df.head())  # Show the first few rows

else:
    print(f"Error: {response.status_code}, {response.text}")


                                               Game    Bookmaker   Market  \
0  Morgan St Bears vs North Carolina Central Eagles  betonlineag  spreads   
1  Morgan St Bears vs North Carolina Central Eagles  betonlineag  spreads   
2  Morgan St Bears vs North Carolina Central Eagles       lowvig  spreads   
3  Morgan St Bears vs North Carolina Central Eagles       lowvig  spreads   
4  Morgan St Bears vs North Carolina Central Eagles   draftkings  spreads   

                            Team  Odds  Point  
0                Morgan St Bears  1.91    2.5  
1  North Carolina Central Eagles  1.91   -2.5  
2                Morgan St Bears  1.94    2.5  
3  North Carolina Central Eagles  1.94   -2.5  
4                Morgan St Bears  1.89    2.5  


In [2]:
print(odds_data)

[{'id': '1ee53fbc0ef8a25a5fd8cfe191c2d54b', 'sport_key': 'basketball_ncaab', 'sport_title': 'NCAAB', 'commence_time': '2026-01-12T23:00:00Z', 'home_team': 'Morgan St Bears', 'away_team': 'North Carolina Central Eagles', 'bookmakers': [{'key': 'betonlineag', 'title': 'BetOnline.ag', 'last_update': '2026-01-12T14:35:04Z', 'markets': [{'key': 'spreads', 'last_update': '2026-01-12T14:35:04Z', 'outcomes': [{'name': 'Morgan St Bears', 'price': 1.91, 'point': 2.5}, {'name': 'North Carolina Central Eagles', 'price': 1.91, 'point': -2.5}]}]}, {'key': 'lowvig', 'title': 'LowVig.ag', 'last_update': '2026-01-12T14:35:51Z', 'markets': [{'key': 'spreads', 'last_update': '2026-01-12T14:35:51Z', 'outcomes': [{'name': 'Morgan St Bears', 'price': 1.94, 'point': 2.5}, {'name': 'North Carolina Central Eagles', 'price': 1.94, 'point': -2.5}]}]}, {'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2026-01-12T14:36:15Z', 'markets': [{'key': 'spreads', 'last_update': '2026-01-12T14:36:14Z', 'outcomes

In [3]:
df.to_csv('odds.csv', index=False)

In [4]:
odds = pd.read_csv('odds.csv')

In [5]:
odds['Team'].unique()

array(['Morgan St Bears', 'North Carolina Central Eagles',
       'Alabama St Hornets', 'Alcorn St Braves', 'American Eagles',
       'Navy Midshipmen', 'Arkansas-Pine Bluff Golden Lions',
       'Prairie View Panthers', 'Bethune-Cookman Wildcats',
       'Southern Jaguars', 'Coppin St Eagles',
       'South Carolina St Bulldogs', 'Delaware St Hornets',
       'Howard Bison', 'Florida A&M Rattlers', 'Grambling St Tigers',
       'Houston Christian Huskies', 'Lamar Cardinals',
       'Incarnate Word Cardinals', 'Stephen F. Austin Lumberjacks',
       'Maryland-Eastern Shore Hawks', 'Norfolk St Spartans',
       'New Orleans Privateers', 'SE Louisiana Lions',
       'East Texas A&M Lions', 'Texas A&M-CC Islanders',
       'McNeese Cowboys', 'Nicholls St Colonels',
       'Northwestern St Demons', 'UT Rio Grande Valley Vaqueros',
       'Alabama A&M Bulldogs', 'Jackson St Tigers',
       'Miss Valley St Delta Devils', 'Texas Southern Tigers'],
      dtype=object)

In [7]:
#Create list of teams with two names
teams = ['Southern Miss Golden Eagles','Maine Black Bears', 'UMass Lowell River Hawks', 'Albany Great Danes', 'Georgia Tech Yellow Jackets', 'Oakland Golden Grizzlies', 'Kent State Golden Flashes'
         ,'Lehigh Mountain Hawks','North Carolina Tar Heels','Rutgers Scarlet Knights','TCU Horned Frogs','Tulsa Golden Hurricane',"Louisiana Ragin' Cajuns",'Nevada Wolf Pack','Arizona St Sun Devils',
         'DePaul Blue Demons','Alabama Crimson Tide','Central Connecticut St Blue Devils','Tennessee Tech Golden Eagles','Texas Tech Red Raiders','Oral Roberts Golden Eagles'
         ,'St. Francis (PA) Red Flash','Marquette Golden Eagles','Minnesota Golden Gophers','California Golden Bears','Notre Dame Fighting Irish','Illinois Fighting Illini','Texas Tech Red Raiders'
         ,'Duke Blue Devils']

In [8]:
# Function to remove the last word
def remove_last_word(text):
    if text == 'Over' or text == 'Under':
        return text
    if text in teams:
        return ' '.join(text.split()[:-2])
    else:
        return ' '.join(text.split()[:-1])

In [9]:
def convert_decimal_to_american(decimal_odds):
    if decimal_odds < 2:
        return -100 / (decimal_odds - 1)
    else:
        return decimal_odds - 1

In [10]:
# Apply the function to the DataFrame column
odds['Team Modified'] = odds['Team'].apply(remove_last_word)

In [11]:
odds['Team Modified'] = odds['Team Modified'].str.replace('State', 'St')
odds['Team Modified'] = odds['Team Modified'].str.replace('St', 'St.')

In [12]:
odds['Implied Probability'] = 1 / odds['Odds']

#Set vegas line column by converting decimal odds to american odds
odds['Vegas Line'] = odds['Odds'].apply(convert_decimal_to_american)

In [13]:
final_odds = odds[['Team Modified', 'Market', 'Bookmaker', 'Point', 'Implied Probability', 'Vegas Line']]
final_odds.to_csv('01-12-2026 Odds.csv', index=False) 

In [14]:
final_odds['Bookmaker'].unique()

array(['betonlineag', 'lowvig', 'draftkings', 'fanduel', 'betmgm',
       'betrivers', 'bovada'], dtype=object)

In [15]:
final_odds

,Team Modified,Market,Bookmaker,Point,Implied Probability,Vegas Line
0,Morgan St.,spreads,betonlineag,2.5,0.523560,-109.890110
1,North Carolina Central,spreads,betonlineag,-2.5,0.523560,-109.890110
2,Morgan St.,spreads,lowvig,2.5,0.515464,-106.382979
3,North Carolina Central,spreads,lowvig,-2.5,0.515464,-106.382979
4,Morgan St.,spreads,draftkings,2.5,0.529101,-112.359551
...,...,...,...,...,...,...
223,Texas Southern,spreads,betmgm,-14.5,0.505051,-102.040816
224,Miss Valley St. Delta,spreads,betrivers,14.5,0.520833,-108.695652
225,Texas Southern,spreads,betrivers,-14.5,0.531915,-113.636364
226,Miss Valley St. Delta,spreads,bovada,14.0,0.523560,-109.890110
